In [ ]:
# Packing

numbers = 1, 2, 3  # Packs this into a tuple
print(numbers)
x, y, z = numbers  # Unpacks each value
print(x, y, z)

a, *b = numbers  # Star unpacking allows 1 variable to take N values
print(a, b)
numbers = a, b
print(numbers)
numbers = a, *b  # Star unpacking allows N variables to unpack
print(numbers)

In [ ]:
# Values are either immutable and act as Copy on Write (string, int, etc.)
# Mutable and pass by reference (most things).
a = 1
b = a
a += 1
print(a, b)

c =[1]
d = c
d.append(2)
print(c, d)


In [ ]:
# Scopes - these are namespaces where variables are looked up by their name. {str : val}.
x = 1
def f():
    y = 2
    def g():
        z = 3
        print(locals())
        print(globals())
    g()
f()

In [ ]:
# To modify a variable out of the local scope you need to declare it as
# coming from another scope.
x = 1
def f():
    global x
    x += 1
f()
print(x)

In [ ]:
# nonlocal means it is not int the local scope nor in the global one.
def f():
    x = 1
    def g():
        nonlocal x
        x = 2
    g()
    return x
f()

In [ ]:
# Late binding can cause capture by reference when we expect capture by value.
fs = []
for i in range(10):
    fs.append(lambda: i)
print(fs[3]())

# Adding indirection forces the variable to be captured by value since the
# ref is temporary.
def create_lambda(i):
    return lambda: i

fs = []
for i in range(10):
    fs.append(create_lambda(i))
print(fs[3]())


In [ ]:
# Walrus operator - like ‘=’ in C which returns the value of the assignment
xs = [1, 2, 3, 4, 5, 6]
if (n := len(xs)) > 5:
    print(f'list is too long: {n}')

In [ ]:
# Conditions return the last value assessed
def greet(name):
    print(f'Hello, {name or "stranger"}')
greet('Matan')
greet('')

In [ ]:
# Compund conditions - works like 'and' statements
# Ternary operator - then_value if condition else else_value
x = 1.1
"true" if 1 < x < 2 else "false"

In [ ]:
# Foreach is native for looping.
# Can iterate a slice which controls the directions and step size.
# Slicing does though create a temporary list with refs to all the values.
# The cost is in the size of list, not the size of the elements.
ll = [1, 2, 3]
for i in ll[::-1]:
    print(i)

# Many operators are baked in though and help read like English.
# This also avoids creation of a temporary list.
for i in reversed(ll):
    print(i)

In [ ]:
# Else can be put at the end of a loop to run iff the loop completes (not break).
for x in [1, 2, 3]:
    for y in [3, 4, 5]:
        print(x, y)
        if x == y:
            break
    else:
        # This means y completed iterating without a break.
        continue
    # This means that the 'else' clause didn't run.
    break

In [ ]:
# Loops can overwrite values.
x = 1
for x in [2]:
    pass
x


In [27]:
# Mutable default arg.
def append(items=[]):
    items.append(1)
    return items

print(append())
print(append())

[1]
[1, 1]


In [58]:
# Star packing works for positional parameters.
def average(*ls):
    return sum(ls) / len(ls)
average(1), average(1, 2, 3)

(1.0, 2.0)

In [44]:
# kwargs - combination of star packing with keyword arguments.
# The double ** signals "take all keyword args that don't match an 
# explicit parameter and build them into a dict of {name : val}"
def f(x, **kwargs):
    print(kwargs)
f(x=1, y=2)

{'y': 2}
1 2


In [41]:
# Double star unpacking also works to destructure a dict into {key : val}
d1 = {'x': 1}
d2 = {'y': 2}
{**d1, **d2}  # If there is a key conflict, the last one wins.

{'x': 1, 'y': 2}

In [49]:
# Placing a '*' without arg name forces all following params to be
# specified by name at the callsite.
def f(x, *, y):
    print(y, x)
f(1, y=2)  # must use `y=`

# Placing a '/' forces all leading params to be specified by position 
# at the callsite.
def g(x, y, /):
    print(x, y)
g(1, 2)  # TypeError if name the args.

2 1
1 2


In [50]:
# Type annotations act as documentation. mypy can be used to check type validity.
def add(a: int, b: int) -> int:
    return a + b
print(add(1, 2))
print(add('Hello', 'World'))

3
HelloWorld


In [57]:
# Functions are first class citizens.

# Can be passed as parameters.
def call_twice(f, *args, **kwargs):
    f(*args, **kwargs)
    f(*args, **kwargs)
call_twice(print, 'Hello, world!')

# Can be return valus:
def create_power(n):
    def power(x):
        # Depends on capturing n by value.
        return x ** n
    return power
square = create_power(2)
cube = create_power(3)
square(3), cube(3)


Hello, world!
Hello, world!


(9, 27)

In [63]:
# Functions can hold fields
def hello():
    hello.runs += 1  # Called with fn name, not self.
    print('Hello, world!')
hello.runs = 0
hello()
hello()
hello.runs

Hello, world!
Hello, world!


2

In [64]:
# Decorators can wrap a function in some additional logic
def double(f):
    def wrapper(*args, **kwargs):
        return f(*args, **kwargs) * 2
    return wrapper

@double
def inc(x):
    return x + 1
inc(1)

4

In [86]:
# 2nd order decorators.
# Making the wrapper transparent (just use functools.wraps).
def wraps(original):
    def wrapper(inner_wrapper):
        inner_wrapper.__name__ = original.__name__
        inner_wrapper.__doc__ = original.__doc__
        return inner_wrapper
    return wrapper

def double(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        return f(*args, **kwargs) * 2
    return wrapper

@double
def inc(x):
    """Increment the input by 1"""
    return x + 1
print(inc.__name__)
print(inc.__doc__)
inc(1)

inc
Increment the input by 1


4

In [83]:
# Decorators can add state like a normal fn returning a fn.
def memoize(f):
    cache = {}
    def wrapper(*args, **kwargs):
        token = args + tuple(kwargs.items())
        if token not in cache:
            cache[token] = f(*args, **kwargs)
        return cache[token]
    return wrapper

@memoize
def fib(n):
    return n if n < 2 else fib(n-1) + fib(n-2)
print(fib(36))

# Run again without memoize.
def fib(n):
    return n if n < 2 else fib(n-1) + fib(n-2)
print(rawfib(36))

14930352
14930352


In [123]:
# Semi parameterized tracing.

import functools

# Log traceing example.
def trace(f=None, /, *, log=print):
    """
    log is a function that takes the log line and flushes it to the correct 
    destination.
    """
    if f is None:
        return lambda f: trace(f, log=log)

    @functools.wraps(f)
    def wrapper(*args, **kwargs):
        call = f'{f.__name__}('
        if args:
            call += ', '.join(repr(arg) for arg in args)
        if kwargs:
            call += ', '.join(f'{key}={value!r}' for key, value in kwargs.items())
        call += ')'
        log(f'enter {call}')
        try:
            result = f(*args, **kwargs)
            log(f'leave {call}: {result!r}')
            return result
        except Exception as error:
            log(f'leave {call} on error: {error}')
            raise
    return wrapper

# Use a single filehandle for any given log file.
# - `handle` is created outside the scope of `write`, so all calls 
#   to a given logger will reuse the same handle.
# - memoize then means that for any given fname, we will return a
#   a ref to the same logger.
@memoize
def logger(fname):
    handle = open(fname, 'a')
    def write(line):
        handle.write(line + '\n')
    return write

@trace(log=logger('/tmp/log.txt'))
def inc(x):
    return x + 1
print(inc.__name__)
inc(1)

@trace
def inc(x):
    return x + 1
print(inc.__name__)
inc(1)

inc
inc
enter inc(1)
leave inc(1): 2


2